In [11]:
import pandas as pd
from PyPDF2 import PdfReader, PdfWriter



In [13]:
def extract_nth_page(input_pdf_path, output_pdf_path, n):

    with open(input_pdf_path, "rb") as file:
        pdf_reader = PdfReader(file)
        
        # Check if n is within the range of available pages
        if n < 1 or n > len(pdf_reader.pages):
            print(f"Error: The PDF does not have a page number {n}.")
            return
        
        # Create a new PDF writer
        pdf_writer = PdfWriter()
        
        # Add the nth page to the writer (adjusting for 0-based index)
        pdf_writer.add_page(pdf_reader.pages[n - 1])
        
        # Save the new PDF
        with open(output_pdf_path, "wb") as output_file:
            pdf_writer.write(output_file)

# Example usage
input_pdf_path = "night_audits.pdf"
output_pdf_path = "one_page.pdf"

extract_nth_page(input_pdf_path, output_pdf_path, 2)